In [7]:
# !wget https://indic-asr-public.objectstore.e2enetworks.net/tamil.zip
!unzip -o -q tamil.zip

In [8]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         124G   50G   75G  40% /
tmpfs            64M     0   64M   0% /dev
/dev/sdc        1.2T  1.2T   51G  96% /home/ubuntu
/dev/root       124G   50G   75G  40% /etc/hosts
shm              64M     0   64M   0% /dev/shm
tmpfs            15G   12K   15G   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs           7.9G     0  7.9G   0% /proc/acpi
tmpfs           7.9G     0  7.9G   0% /proc/scsi
tmpfs           7.9G     0  7.9G   0% /sys/firmware


In [10]:
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-tamil-large-v3-timestamp/resolve/main/prepared-tamil.jsonl
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-tamil-large-v3-timestamp/resolve/main/prepared-tamil-en.jsonl
# !wget https://huggingface.co/datasets/mesolitica/pseudolabel-tamil-large-v3-timestamp/resolve/main/prepared-tamil-ms.jsonl

In [11]:
!ls newsonair_v5

tamil


In [12]:
from glob import glob
from tqdm import tqdm
import os

In [13]:
from collections import defaultdict
import json

mapping_en = dict()

with open('prepared-tamil-en.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping_en[l['audio_filename']] = l['new_text']

mapping_ms = dict()

with open('prepared-tamil-ms.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping_ms[l['audio_filename']] = l['new_text']

In [16]:
!ls -lh /home/ubuntu/newsonair_v5/tamil/NSD-Tamil-Tamil-0715-0725-20195208161/sent_11.wav

-rw-r--r-- 1 ubuntu ubuntu 234K Jun  5  2022 /home/ubuntu/newsonair_v5/tamil/NSD-Tamil-Tamil-0715-0725-20195208161/sent_11.wav


In [18]:
import json
import os
from tqdm import tqdm
from streaming import MDSWriter

columns = {
    'new_text': 'str',
    'audio_filename': 'str',
    'new_text_en': 'str',
    'new_text_ms': 'str',
}
hashes = 'sha1', 'xxh64'

with MDSWriter(out='tamil-mosaic', columns=columns, compression=None, hashes=hashes) as out:
    with open('prepared-tamil.jsonl') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            if not os.path.exists(l['audio_filename']):
                continue
            l['new_text_en'] = mapping_en.get(l['audio_filename'], '')
            l['new_text_ms'] = mapping_ms.get(l['audio_filename'], '')
            out.write(l)

532262it [00:24, 22070.76it/s]


In [19]:
l

{'new_text': '<|startoftranscript|><|ta|><|transcribe|><|0.00|> இதன்படி அனைத்து மருத்துமனைகள் கோவிட் சிகிச்சி மயங்கள் விழிட்டவே,<|3.40|><|3.74|> சிகிச்சிக்கான கட்டணத் தொகையை இரண்டு லிச்சம் ரூபாய்க்கு மேல் ரொக்கமாக பெற்றுக்கொள்ள,<|7.78|><|endoftext|>',
 'audio_filename': '/home/ubuntu/newsonair_v5/tamil/Regional-Tiruchirapalli-Tamil-1345-202158145135/sent_67.wav',
 'new_text_en': '',
 'new_text_ms': ''}